In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,23676
2,Huelva,Confirmados PDIA 14 días,3345
3,Huelva,Tasa PDIA 14 días,"651,8307773252528"
4,Huelva,Confirmados PDIA 7 días,1076
5,Huelva,Total Confirmados,23907
6,Huelva,Curados,12588
7,Huelva,Fallecidos,234


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  23676.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6271.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 564 personas en los últimos 7 días 

Un positivo PCR cada 180 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,23676.0,1076.0,3345.0,513170.0,209.677105,651.830777,176.0
Huelva-Costa,14394.0,667.0,2000.0,289548.0,230.359042,690.731761,121.0
Huelva (capital),6271.0,255.0,797.0,143837.0,177.284009,554.099432,51.0
Condado-Campiña,6558.0,320.0,1030.0,156231.0,204.824907,659.280168,49.0
Isla Cristina,1942.0,99.0,241.0,21393.0,462.768195,1126.536718,19.0
Cartaya,862.0,24.0,126.0,20083.0,119.504058,627.396305,14.0
Ayamonte,1161.0,81.0,230.0,21104.0,383.813495,1089.840788,13.0
Sierra de Huelva-Andévalo Central,2309.0,49.0,195.0,67391.0,72.710006,289.356145,12.0
Lepe,1903.0,79.0,275.0,27880.0,283.357245,986.370158,11.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villablanca,159.0,26.0,59.0,2885.0,901.213172,2045.060659,1.0
Berrocal,19.0,3.0,5.0,303.0,990.099010,1650.165017,0.0
Paymogo,72.0,4.0,16.0,1162.0,344.234079,1376.936317,1.0
Villalba del Alcor,436.0,12.0,45.0,3366.0,356.506239,1336.898396,0.0
Cabezas Rubias,28.0,1.0,8.0,706.0,141.643059,1133.144476,0.0
Isla Cristina,1942.0,99.0,241.0,21393.0,462.768195,1126.536718,19.0
Ayamonte,1161.0,81.0,230.0,21104.0,383.813495,1089.840788,13.0
Bollullos Par del Condado,555.0,41.0,153.0,14387.0,284.979495,1063.460068,6.0
Alájar,15.0,2.0,8.0,759.0,263.504611,1054.018445,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Paterna del Campo,110.0,1.0,23.0,3457.0,28.926815,665.316749,0.0,0.043478
San Bartolomé de la Torre,188.0,4.0,39.0,3761.0,106.354693,1036.958256,1.0,0.102564
Calañas,169.0,2.0,17.0,2768.0,72.254335,614.161850,0.0,0.117647
Cabezas Rubias,28.0,1.0,8.0,706.0,141.643059,1133.144476,0.0,0.125000
Niebla,156.0,2.0,14.0,4116.0,48.590865,340.136054,0.0,0.142857
Zalamea la Real,85.0,3.0,18.0,3054.0,98.231827,589.390963,1.0,0.166667
Nerva,147.0,2.0,12.0,5169.0,38.692204,232.153221,0.0,0.166667
Escacena del Campo,128.0,2.0,11.0,2287.0,87.450809,480.979449,1.0,0.181818
Valverde del Camino,418.0,10.0,54.0,12750.0,78.431373,423.529412,5.0,0.185185
